In [ ]:
# Standard libraries
import numpy as np
import time
import os

# Computer Vision and Image Processing libraries
import cv2
import torch
import kornia as K

# Specific modules for feature extraction and visualization
import kornia.feature as KF
from kornia_moons.feature import *
from kornia_moons.viz import *

# Utility and progress bar
import tqdm

# Visualization libraries
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
# Process the file
with open("../image_list_hpatches_sequences.txt", 'r') as f:
    lines = [os.path.join("..", line.strip()) for line in f.readlines()]

In [ ]:
num_features = 2048
keynet = KF.KeyNetAffNetHardNet(num_features, True).eval().to(device)
output_extension = ".keynet"

durations = []

for path in lines:

    image = K.io.load_image(path, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    image_g = K.color.rgb_to_grayscale(image)

    starttime = time.time()
    
    with torch.no_grad():
        lafs, resp, descs = keynet.forward(image_g)

    total = round((time.time() - starttime), 2)
    durations.append(total)
    
    with open(path + output_extension, 'wb') as output_file:
        np.savez(
            output_file,
            keypoints=lafs[0, :, :, 2].cpu().numpy(),
            descriptors=descs[0].cpu().numpy(),
            )
        
    print(f"Processed {path} in {total} seconds")
